In [9]:
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    if total_rule/total_records<0.4:
        return (999,);

    avg_confidence = np.sum(rule_confidences) / count+0.3*(entropy_D-entropy_DB)+1*(total_rule/total_records)

    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    return hof



def bestIndividual(hof,df):

    def SVM(trainX,trainY,testX,testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=200)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY)/testY.shape[0]
        return precision
    maxaccuracy=0.0
    for ind in hof:
        df = pd.read_csv('data/German2.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        trainX, testX, trainY, testY = train_test_split(X,Y,test_size=0.33,random_state=1)
        score=SVM(trainX,trainY,testX,testY)
        if(score>maxaccuracy):
            maxaccuracy=score
            print(maxaccuracy)
            best_feature=selected_features
            print(best_feature)
    return maxaccuracy,best_feature


if __name__ == '__main__':
    # get dataframe path, population number and generation number from command-line argument
    # dataframePath, n_pop, n_gen = getArguments()
    n_pop=50
    n_gen=60
    # read dataframe from csv
    df = pd.read_csv('data/German2.csv')
    # encode labels column to numbers
    y = df['target']
    feature_columns = [ col for col in df.columns if col not in ['id','target']]
    X = df[feature_columns]
    individual = [1 for i in range(len(X.columns))]
    getFitness(df,'target',individual)
    print("Accuracy with all features: \t" +
          str(getFitness(df,'target',individual)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(df,'target',n_pop, n_gen)

Accuracy with all features: 	(1.2644973022998294,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min   
0  	50    	1.28546	1.2645
1  	29    	1.26854	1.2645
2  	36    	1.2657 	1.2645
3  	21    	1.26499	1.2645
4  	31    	1.26471	1.2645
5  	31    	1.26453	1.2645
6  	30    	1.26463	1.2645
7  	31    	1.26478	1.2645
8  	33    	1.26455	1.2645
9  	23    	1.26451	1.2645
10 	33    	1.26454	1.2645
11 	34    	1.26456	1.2645
12 	33    	1.26451	1.2645
13 	24    	1.26455	1.2645
14 	36    	1.26453	1.2645
15 	33    	1.26459	1.2645
16 	32    	1.26451	1.2645
17 	37    	1.26453	1.2645
18 	36    	1.26455	1.2645
19 	35    	1.2646 	1.2645
20 	34    	1.26452	1.2645
21 	29    	1.26457	1.2645
22 	20    	1.26457	1.2645
23 	26    	1.26461	1.2645
24 	32    	1.26453	1.2645
25 	31    	1.26451	1.2645
26 	29    	1.26455	1.2645
27 	28    	1.26455	1.2645
28 	28    	1.26455	1.2645
29 	34    	1.26473	1.2645
30 	31    	1.26455	1.2645
31 	27    	1.26454	1.2645
32 	29    	1.26452	1.2645
33 	31    	1.26455	1.2645
34 	33    	1.2645 	1.2645
35 	30    	1.26451	1.2645
36 	30    	1.26451	1.2645
37 	28    	1

In [10]:
accuracy,individual = bestIndividual(hof,df)
print(accuracy)
print(individual)
    # select the best individual

    # print('Best Accuracy: \t' + str(accuracy))
    # print('Number of Features in Subset: \t' + str(individual.count(1)))
    # print('Individual: \t\t' + str(individual))
    # print('Feature Subset\t: ' + str(header))
    #
    # print('\n\ncreating a new classifier with the result')

    # read dataframe from csv one more time
    # df = pd.read_csv(dataframePath, sep=',')
    #
    # # with feature subset
    # X = df[header]
    #
    # clf = LogisticRegression()
    #
    # scores = cross_val_score(clf, X, y, cv=5)
    # print("Accuracy with Feature Subset: \t" + str(avg(scores)) + "\n")


0.696969696969697
['status_account', 'duration', 'credit_history', 'purpose', 'svaing_account', 'present_emp', 'personal_status', 'residence_info', 'property', 'age', 'inst_plans', 'job', 'dependents']
0.7424242424242424
['status_account', 'duration', 'credit_history', 'purpose', 'amount', 'svaing_account', 'present_emp', 'personal_status', 'other_debtors', 'residence_info', 'property', 'job', 'telephone', 'foreign_worker']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

0.7515151515151515
['status_account', 'duration', 'purpose', 'amount', 'svaing_account', 'present_emp', 'personal_status', 'residence_info', 'property', 'age', 'inst_plans', 'job', 'telephone']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

0.7606060606060606
['status_account', 'duration', 'purpose', 'amount', 'svaing_account', 'present_emp', 'personal_status', 'other_debtors', 'residence_info', 'property', 'age', 'inst_plans', 'job', 'telephone']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

0.7606060606060606
['status_account', 'duration', 'purpose', 'amount', 'svaing_account', 'present_emp', 'personal_status', 'other_debtors', 'residence_info', 'property', 'age', 'inst_plans', 'job', 'telephone']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv